# Membership Inference over Diffusion-models-based Synthetic Tabular Data (MIDST) Challenge @ SaTML 2025.

## White Box Single Table Competition
Welcome to the MIDST challenge!

The MIDST challenge is a multi-track competition aiming to quantitatively evaluate the privacy of synthetic tabular data generated by diffusion models, with a specific focus on its resistance to membership inference attacks (MIAs).

This competition focuses on White Box MIA on tabular diffusion models trained on a single table transaction dataset. The schema of the transaction dataset is as follows:
| trans_id | account_id | trans_date | trans_type | operation | amount  | balance  | k_symbol | bank | account |
|----------|------------|------------|------------|-----------|---------|----------|----------|------|---------|
| integer  | integer    | integer    | integer    | integer   | float   | float    | integer  | integer | integer |

MIA will be performed over two state-of-the-art methods [TabSyn](https://arxiv.org/pdf/2310.09656) and [TabDDPM](https://arxiv.org/pdf/2209.15421). A collection of TabSyn and TabDDPM models will be trained on random subsets of the transaction dataset. The goal is to create an approach (MIA) that can distinguish between samples used to train a model (train data) and other data randomly sampled from the transaction dataset (holdout data) given the model and it's output synthetic data. The `final` set includes 20 models, each with its own set of challenge points (ie train and holdout data), to evaluate solutions on. To facilitate designing an attack, 30 `train` models are provided with comprehensive information about the model, training data and output synthetic data. Additionally, 20 `dev` models are provided to assist in evaluating the effectiveness of attacks prior to making a final submission to the `final` set. Participants can choose to perform MIA over one of or both TabSyn and TabDDPM. In the case of both, the attack that obtains the highest score will be used to rank the submission. A high level summary of the competition is below:
![wbox_diagram_final](https://github.com/user-attachments/assets/2ebb5eed-a6e3-433a-8769-4310b7fbc822)

This notebook will walk you through the process of creating and packaging a submission to the white box single table challenge.

## Package Imports and Evironment Setup

Ensure that you have installed the proper dependenices to run the notebook. The environment installation instructions are available [here](https://github.com/VectorInstitute/MIDSTModels/tree/main/starter_kits). Now that we have verfied we have the proper packages installed, lets import them and define global variables:

In [2]:
import csv
import os
import random
import zipfile

from pathlib import Path
from functools import partial
from typing import Callable, Any

import numpy as np
import torch

from tqdm.notebook import tqdm
from data import get_challenge_points, get_challenge_labels
from metrics import get_tpr_at_fpr



In [3]:
%cd ..

c:\Users\ksush\attacks\MIDSTModels


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from midst_models.single_table_TabDDPM.complex_pipeline import tabddpm_whitebox_load_pretrained

In [5]:
TABDDPM_DATA_DIR = "tabddpm_white_box"
TABSYN_DATA_DIR = "tabsyn_white_box"

In [6]:

def get_FLAGS():
    def FLAGS(x): return x
    FLAGS.T = 5000
    FLAGS.ch = 128
    FLAGS.ch_mult = [1, 2, 2, 2]
    FLAGS.attn = [1]
    FLAGS.num_res_blocks = 2
    FLAGS.dropout = 0.1
    FLAGS.beta_1 = 0.001
    FLAGS.beta_T = 0.2

    return FLAGS

FLAGS = get_FLAGS()


In [12]:
import torch
import numpy as np
import logging
import csv
import os
from typing import Dict, Type
from torch.nn.functional import normalize
import midst_models.attack.components as components

class EpsGetter(components.EpsGetter):
    def __call__(self, xt: torch.Tensor, condition: torch.Tensor = None, noise_level=None, t: int = None) -> torch.Tensor:
        # Access the diffusion model from the dictionary structure
        model = self.model[(None, 'trans')]['diffusion']

        t = torch.ones([xt.shape[0]], device=xt.device).long() * t
        return model._denoise_fn(xt, timesteps=t)

ATTACKERS: Dict[str, Type[components.DDIMAttacker]] = {
    "PIA": components.PIA,
    "PIAN": components.PIAN,
}


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def attack(base_dir, attacker_name="PIA", attack_num=30, interval=10, seed=0):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    # logger.addHandler(RichHandler())

    logger.info("loading the attacked model...")

    # Initialize attacker
    phases = ["train"]
    # phases = ["dev", "final"]

    logger.info("attack start...")
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
            print("path",path)
            model = tabddpm_whitebox_load_pretrained(path)
            attacker = ATTACKERS[attacker_name](
                torch.from_numpy(np.linspace(FLAGS.beta_1, FLAGS.beta_T, FLAGS.T)).to(DEVICE), interval, attack_num, EpsGetter(model), lp=1)

            # Move the points to GPU
            challenge_points = get_challenge_points(path)

            # Get raw predictions
            raw_predictions = torch.stack([attacker(cp.to(DEVICE).float()) for cp in challenge_points])

            # Convert to binary predictions based on membership
            # First normalize the predictions to [0,1] range
            normalized_preds = []
            for pred_batch in raw_predictions:
                print("pred_batch", pred_batch)

                # Convert to binary predictions using a threshold
                binary_preds = (pred_batch >= 1000000).float()
                print("binary_preds", binary_preds)
                normalized_preds.append(binary_preds)
            
            # Stack all normalized predictions
            final_predictions = torch.stack(normalized_preds)
            predictions_cpu = final_predictions.cpu().detach().numpy()
            
            # Verify predictions are binary (0 or 1)
            # assert np.all(np.logical_or(predictions_cpu == 0, predictions_cpu == 1)), "Predictions are not binary"

            # Write predictions to file
            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
                # Write each value in a separate row
                for value in predictions_cpu.squeeze():
                    writer.writerow([value])

            # # Optional logging of sample predictions
            # print(f"raw_predictions.shape: {raw_predictions.shape}")
            # print(f"Sample binary predictions: {predictions_cpu.squeeze()[:5]}")
                

In [13]:
attack(base_dir="tabddpm_white_box",
        attacker_name="PIA",
        attack_num=2,
        interval=200,)

path tabddpm_white_box\train\tabddpm_1
Checkpoint found, loading...
distance tensor([7.6280e+08, 7.4449e+08], grad_fn=<SumBackward1>)
distance tensor([975380.2500, 935019.7500], grad_fn=<SumBackward1>)
distance tensor([936701.3750, 923406.0625], grad_fn=<SumBackward1>)
distance tensor([1.7718e+09, 1.7291e+09], grad_fn=<SumBackward1>)
distance tensor([730300.7500, 699821.3125], grad_fn=<SumBackward1>)
distance tensor([8.7431e+08, 8.5331e+08], grad_fn=<SumBackward1>)
distance tensor([310850.1875, 299768.7188], grad_fn=<SumBackward1>)
distance tensor([3.9159e+08, 3.8167e+08], grad_fn=<SumBackward1>)
distance tensor([800454.0625, 764284.5625], grad_fn=<SumBackward1>)
distance tensor([1383061.7500, 1326067.0000], grad_fn=<SumBackward1>)
distance tensor([660714.6875, 637320.6250], grad_fn=<SumBackward1>)


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([796810.1250, 748343.8125], grad_fn=<SumBackward1>)
distance tensor([1187517.0000, 1126613.2500], grad_fn=<SumBackward1>)
distance tensor([734222.6875, 708438.8125], grad_fn=<SumBackward1>)
distance tensor([989279.0625, 945394.0000], grad_fn=<SumBackward1>)
distance tensor([1392562.5000, 1335300.0000], grad_fn=<SumBackward1>)
distance tensor([1046592.9375,  977849.3750], grad_fn=<SumBackward1>)
distance tensor([2984272.5000, 2883156.7500], grad_fn=<SumBackward1>)
distance tensor([1534094.7500, 1470424.0000], grad_fn=<SumBackward1>)
distance tensor([1.6886e+09, 1.6482e+09], grad_fn=<SumBackward1>)
distance tensor([3.8888e+08, 3.7934e+08], grad_fn=<SumBackward1>)
distance tensor([1759251.7500, 1694862.7500], grad_fn=<SumBackward1>)
distance tensor([503908.7812, 481649.1562], grad_fn=<SumBackward1>)
distance tensor([598963.5000, 570410.0625], grad_fn=<SumBackward1>)
distance tensor([1198459.7500, 1140708.8750], grad_fn=<SumBackward1>)
distance tensor([1.8177e+09, 1.7740e+0

C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([936816.8750, 971221.3125], grad_fn=<SumBackward1>)
distance tensor([1094056.7500, 1107309.5000], grad_fn=<SumBackward1>)
distance tensor([1529910.7500, 1578061.1250], grad_fn=<SumBackward1>)
distance tensor([972851.3125, 992482.3125], grad_fn=<SumBackward1>)
distance tensor([1028789.3125, 1020474.1250], grad_fn=<SumBackward1>)
distance tensor([5.8776e+08, 5.2727e+08], grad_fn=<SumBackward1>)
distance tensor([1141982.7500, 1095764.7500], grad_fn=<SumBackward1>)
distance tensor([4.6953e+08, 4.2126e+08], grad_fn=<SumBackward1>)
distance tensor([1360341.7500, 1374736.5000], grad_fn=<SumBackward1>)
distance tensor([8.2031e+08, 7.3586e+08], grad_fn=<SumBackward1>)
distance tensor([1707842.7500, 1729877.1250], grad_fn=<SumBackward1>)
distance tensor([50166484., 44953192.], grad_fn=<SumBackward1>)
distance tensor([950984.4375, 960485.3125], grad_fn=<SumBackward1>)
distance tensor([1342484.0000, 1358875.1250], grad_fn=<SumBackward1>)
distance tensor([686454.0000, 690882.6875], 

...

distance tensor([179777.4844, 135556.9062], grad_fn=<SumBackward1>)
distance tensor([144835.2188,  92412.3750], grad_fn=<SumBackward1>)
distance tensor([332088.6250, 237562.3906], grad_fn=<SumBackward1>)
distance tensor([452220.3125, 270309.2500], grad_fn=<SumBackward1>)
distance tensor([1000153.3125,  757907.0000], grad_fn=<SumBackward1>)
distance tensor([1.0718e+09, 9.2143e+08], grad_fn=<SumBackward1>)
distance tensor([354492.4688, 252797.7031], grad_fn=<SumBackward1>)
distance tensor([2.8025e+08, 2.4118e+08], grad_fn=<SumBackward1>)
distance tensor([1.4938e+08, 1.2873e+08], grad_fn=<SumBackward1>)
distance tensor([1.4559e+09, 1.2518e+09], grad_fn=<SumBackward1>)
distance tensor([205800.0938, 155876.9688], grad_fn=<SumBackward1>)
distance tensor([1.3784e+09, 1.1852e+09], grad_fn=<SumBackward1>)
distance tensor([329784.9375, 197245.7969], grad_fn=<SumBackward1>)
distance tensor([227513.5469, 172241.1250], grad_fn=<SumBackward1>)
distance tensor([283240.5938, 175782.2656], grad_fn=<Sum

C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([1.2066e+09, 1.0559e+09], grad_fn=<SumBackward1>)
distance tensor([3203971.5000, 3115376.2500], grad_fn=<SumBackward1>)
distance tensor([265078.1562, 308940.4375], grad_fn=<SumBackward1>)
distance tensor([3427035.5000, 3322174.5000], grad_fn=<SumBackward1>)
distance tensor([971196.1250, 951486.5000], grad_fn=<SumBackward1>)
distance tensor([533563.5000, 545688.6250], grad_fn=<SumBackward1>)
distance tensor([406262.1250, 487682.3750], grad_fn=<SumBackward1>)
distance tensor([626323.9375, 640823.7500], grad_fn=<SumBackward1>)
distance tensor([1.2020e+09, 1.0516e+09], grad_fn=<SumBackward1>)
distance tensor([1364511.3750, 1223563.1250], grad_fn=<SumBackward1>)
distance tensor([1275044.1250, 1258499.3750], grad_fn=<SumBackward1>)
distance tensor([1326113.7500, 1282854.8750], grad_fn=<SumBackward1>)
distance tensor([4.7089e+08, 4.1190e+08], grad_fn=<SumBackward1>)
distance tensor([1612508.7500, 1419855.2500], grad_fn=<SumBackward1>)
distance tensor([2309409.0000, 2108645.500

C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


distance tensor([6206933.0000, 7008030.5000], grad_fn=<SumBackward1>)
distance tensor([2872434.2500, 3229863.2500], grad_fn=<SumBackward1>)
distance tensor([76759008., 58424368.], grad_fn=<SumBackward1>)
distance tensor([7.0109e+08, 5.3189e+08], grad_fn=<SumBackward1>)
distance tensor([ 8918920., 10169654.], grad_fn=<SumBackward1>)
distance tensor([7.4920e+08, 5.6840e+08], grad_fn=<SumBackward1>)
distance tensor([2655451., 2985630.], grad_fn=<SumBackward1>)
distance tensor([6165261.5000, 6998420.0000], grad_fn=<SumBackward1>)
distance tensor([8231654., 9421332.], grad_fn=<SumBackward1>)
distance tensor([12417890., 14108675.], grad_fn=<SumBackward1>)
distance tensor([5.5557e+08, 4.2150e+08], grad_fn=<SumBackward1>)
distance tensor([6613314.5000, 7516220.5000], grad_fn=<SumBackward1>)
distance tensor([6001737.5000, 6781532.5000], grad_fn=<SumBackward1>)
distance tensor([5948056., 6718833.], grad_fn=<SumBackward1>)
distance tensor([7055053., 7986027.], grad_fn=<SumBackward1>)
distance ten

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [14]:
import numpy as np
from sklearn.metrics import roc_auc_score
import os
from typing import List, Tuple

def safe_load_data(filepath: str, is_prediction: bool = True) -> np.ndarray:
    """
    Safely load data from CSV files with error handling and debugging.
    
    Args:
        filepath: Path to the CSV file
        is_prediction: Whether this is a prediction file (True) or label file (False)
    
    Returns:
        numpy.ndarray: Loaded and validated data
    """
    try:
        if is_prediction:
            # Read the file line by line and parse each array
            predictions = []
            with open(filepath, 'r') as f:
                for line in f:
                    # Remove brackets and split by spaces
                    clean_line = line.strip().replace('[', '').replace(']', '')
                    # Convert space-separated strings to floats
                    row = np.array([float(x) for x in clean_line.split()])
                    predictions.append(row)
            return np.array(predictions)
        else:
            # For label files, skip header and use numpy
            return np.loadtxt(filepath, skiprows=1)
            
    except Exception as e:
        print(f"Error loading {filepath}: {str(e)}")
        print(f"File contents (first few lines):")
        with open(filepath, 'r') as f:
            print(f.read(500))
        raise

def get_tpr_at_fpr(y_true: np.ndarray, y_score: np.ndarray, target_fpr: float = 0.1) -> float:
    """Calculate TPR at a specific FPR threshold."""
    # Input validation
    if not isinstance(y_true, np.ndarray) or not isinstance(y_score, np.ndarray):
        raise TypeError("Inputs must be numpy arrays")
    if y_true.shape != y_score.shape:
        raise ValueError(f"Shape mismatch: y_true {y_true.shape} != y_score {y_score.shape}")
    
    # Sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]
    
    n_neg = np.sum(y_true == 0)
    n_pos = np.sum(y_true == 1)
    
    # Handle edge cases
    if n_neg == 0 or n_pos == 0:
        print("Warning: Found no positive or negative samples")
        return 0.0
        
    tpr = np.cumsum(y_true) / n_pos
    fpr = np.cumsum(1 - y_true) / n_neg
    
    for i in range(len(fpr)):
        if fpr[i] >= target_fpr:
            return tpr[i]
    return 1.0

def evaluate_membership_inference(base_dirs: List[str]) -> Tuple[float, float]:
    """
    Evaluate membership inference attack results across multiple directories.
    
    Args:
        base_dirs: List of base directories containing prediction files
        
    Returns:
        tuple: (best_tpr_at_fpr, best_auc)
    """
    tpr_at_fpr_list = []
    auc_list = []
    
    for base_dir in base_dirs:
        predictions = []
        solutions = []
        root = os.path.join(base_dir, "train")
        
        if not os.path.exists(root):
            print(f"Warning: Directory not found: {root}")
            continue
            
        model_folders = sorted(os.listdir(root), key=lambda d: int(d.split('_')[1]))
        if not model_folders:
            print(f"Warning: No model folders found in {root}")
            continue
        
        # Load and process all predictions and solutions
        for model_folder in model_folders:
            path = os.path.join(root, model_folder)
            pred_path = os.path.join(path, "prediction.csv")
            label_path = os.path.join(path, "challenge_label.csv")
            
            if not (os.path.exists(pred_path) and os.path.exists(label_path)):
                print(f"Warning: Missing files in {path}")
                continue
            
            try:
                # Load predictions
                pred = safe_load_data(pred_path, is_prediction=True)
                predictions.append(pred)
                
                # Load ground truth
                solution = safe_load_data(label_path, is_prediction=False)
                if solution.ndim == 0:
                    solution = solution.reshape(1)
                solutions.append(solution)
                
            except Exception as e:
                print(f"Error processing folder {model_folder}: {str(e)}")
                continue
        
        if not predictions or not solutions:
            print(f"Warning: No valid data found in {base_dir}")
            continue
            
        try:
            # Concatenate all predictions and solutions
            predictions = np.concatenate(predictions)
            solutions = np.concatenate(solutions)
            
            print(f"\nData shapes for {os.path.basename(base_dir)}:")
            print(f"Predictions shape: {predictions.shape}")
            print(f"Solutions shape: {solutions.shape}")
            
            # Calculate metrics for each attacker's predictions
            num_attackers = predictions.shape[1]
            for attacker_idx in range(num_attackers):
                attacker_preds = predictions[:, attacker_idx]
                
                # Basic data validation
                if np.any(np.isnan(attacker_preds)):
                    print(f"Warning: NaN values found in predictions for attacker {attacker_idx}")
                    continue
                    
                # Calculate metrics
                tpr_at_fpr = get_tpr_at_fpr(solutions, attacker_preds)
                tpr_at_fpr_list.append(tpr_at_fpr)
                
                try:
                    auc = roc_auc_score(solutions, attacker_preds)
                    auc_list.append(auc)
                except ValueError as e:
                    print(f"Warning: Could not calculate AUC for attacker {attacker_idx}: {e}")
                    continue
                
                print(f"{os.path.basename(base_dir)} Attacker {attacker_idx + 1}:")
                print(f"  TPR at FPR==10%: {tpr_at_fpr:.4f}")
                print(f"  AUC: {auc:.4f}")
                
        except Exception as e:
            print(f"Error processing directory {base_dir}: {str(e)}")
            continue
    
    # Get best scores
    final_tpr_at_fpr = max(tpr_at_fpr_list) if tpr_at_fpr_list else 0.0
    final_auc = max(auc_list) if auc_list else 0.0
    
    print(f"\nBest scores across all attackers:")
    print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr:.4f}")
    print(f"Final Train Attack AUC: {final_auc:.4f}")
    
    return final_tpr_at_fpr, final_auc

In [ ]:
base_dirs = [TABDDPM_DATA_DIR]
final_tpr, final_auc = evaluate_membership_inference(base_dirs)


Data shapes for tabddpm_white_box:
Predictions shape: (6000, 2)
Solutions shape: (6000,)
tabddpm_white_box Attacker 1:
  TPR at FPR==10%: 0.1017
  AUC: 0.4970
tabddpm_white_box Attacker 2:
  TPR at FPR==10%: 0.1033
  AUC: 0.4985

Best scores across all attackers:
Final Train Attack TPR at FPR==10%: 0.1033
Final Train Attack AUC: 0.4985


## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaBench. Importantly, we create a single zip file for dev and final. The structure of the submission is as follows:

```
└── root_folder
    ├── tabsyn_white_box
    │   ├── dev
    │   │   └── tabsyn_#
    │   │       └── prediction.csv
    │   └── final
    │       └── tabsyn_#
    │           └── prediction.csv
    └── tabddpm_white_box
        ├── dev 
        │   └── tabddpm_#
        │       └── prediction.csv
        └── final 
            └── tabddpm_# 
                └── prediction.csv
```
**Note:** The `root_folder` can have any name but it is important all of the subdirectories follow the above structure and naming conventions. 

If a participant is looking to submit an attack for only one of TabSyn and TabDDPM, they can simply omit the other directory (ie `tabddpm_white_box` or `tabsyn_white_box` from the root_folder).

In [11]:
import os
import zipfile
import pandas as pd
import numpy as np
import re  # For fixing malformed lists

def parse_numpy_array(array_str):
    """Fix NumPy-style array formatting and convert it to a Python list."""
    array_str = array_str.strip()  # Remove leading/trailing spaces
    array_str = re.sub(r'\s+', ',', array_str)  # Replace spaces with commas
    array_str = array_str.replace("[,", "[").replace(",]", "]")  # Fix misplaced commas
    return eval(array_str)  # Convert string to list

with zipfile.ZipFile("white_box_single_table_submission.zip", 'w') as zipf:
    for phase in ["dev", "final"]:
        for base_dir in [TABDDPM_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): 
                    continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Load CSV
                    df = pd.read_csv(file, header=None)

                    # Convert NumPy-style arrays to proper lists
                    df = df[0].apply(lambda x: parse_numpy_array(x) if isinstance(x, str) else x)

                    # Compute mean for each row
                    df_mean = df.apply(lambda x: np.mean(x) if isinstance(x, list) else x)

                    # Save the new CSV
                    df_mean.to_csv(file, index=False, header=False)

                    # Add to ZIP
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}.")


The generated white_box_single_table_submission.zip can be directly submitted to the dev phase in the CodaBench UI. Although this submission contains your predictions for both the dev and final set, you will only receive feedback on your predictions for the dev phase. The predictions for the final phase will be evaluated once the competiton ends using the most recent submission to the dev phase.